<a href="https://colab.research.google.com/github/daniel-usp/MachineLearning/blob/main/04%20-%20DecisionTrees/Prompt_DecisionTreeClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:


def importar_bibliotecas():
    """
    Importa todas as bibliotecas necessárias para o projeto.
    """
    import pandas as pd
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.tree import DecisionTreeClassifier, export_graphviz
    from sklearn.metrics import accuracy_score, classification_report
    import matplotlib.pyplot as plt
    import graphviz

def carregar_dados(url):
    """
    Carrega os dados a partir de uma URL específica.

    Parameters:
    url (str): URL do arquivo Excel contendo os dados.

    Returns:
    pd.DataFrame: DataFrame contendo os dados carregados.
    """
    try:
        dados = pd.read_excel(url)
        print("Dados carregados com sucesso.")
        return dados
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")

def preprocessar_dados(dados, coluna_alvo):
    """
    Separa os dados em variáveis independentes (X) e variável dependente (y).

    Parameters:
    dados (pd.DataFrame): DataFrame contendo os dados.
    coluna_alvo (str): Nome da coluna alvo.

    Returns:
    tuple: Contendo X e y.
    """
    y = dados[coluna_alvo]
    X = dados.drop(coluna_alvo, axis=1)
    print("Dados pré-processados.")
    return X, y

def dividir_dados(X, y, teste_size=0.3, random_state=42):
    """
    Divide os dados em conjuntos de treino e teste.

    Parameters:
    X (pd.DataFrame): Variáveis independentes.
    y (pd.Series): Variável dependente.
    teste_size (float): Proporção dos dados para teste.
    random_state (int): Semente para reprodutibilidade.

    Returns:
    tuple: X_train, X_test, y_train, y_test
    """
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=teste_size, random_state=random_state)
    print(f"Dados divididos: {len(X_train)} para treino e {len(X_test)} para teste.")
    return X_train, X_test, y_train, y_test

def ajustar_hiperparametros(X_train, y_train):
    """
    Realiza a busca em grade para encontrar os melhores hiperparâmetros.

    Parameters:
    X_train (pd.DataFrame): Dados de treino.
    y_train (pd.Series): Rótulos de treino.

    Returns:
    dict: Melhores parâmetros encontrados.
    float: Melhor acurácia durante a validação cruzada.
    """
    param_grid = {
        'max_depth': list(range(1, 11)) + [15],
        'splitter': ['best', 'random']
    }

    grid_search = GridSearchCV(
        DecisionTreeClassifier(random_state=42),
        param_grid,
        cv=5,
        scoring='accuracy'
    )
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    best_accuracy = grid_search.best_score_

    print(f"Melhores parâmetros encontrados: {best_params}")
    print(f"Melhor acurácia na validação cruzada: {best_accuracy:.2f}")

    return best_params, best_accuracy

def treinar_modelo(X_train, y_train, params):
    """
    Treina o modelo de Árvore de Decisão com os melhores parâmetros.

    Parameters:
    X_train (pd.DataFrame): Dados de treino.
    y_train (pd.Series): Rótulos de treino.
    params (dict): Parâmetros para o modelo.

    Returns:
    DecisionTreeClassifier: Modelo treinado.
    """
    tree = DecisionTreeClassifier(
        max_depth=params['max_depth'],
        splitter=params['splitter'],
        random_state=42
    )
    tree.fit(X_train, y_train)
    print("Modelo treinado com sucesso.")
    return tree

def avaliar_modelo(modelo, X_test, y_test):
    """
    Avalia o modelo treinado nos dados de teste.

    Parameters:
    modelo (DecisionTreeClassifier): Modelo treinado.
    X_test (pd.DataFrame): Dados de teste.
    y_test (pd.Series): Rótulos de teste.

    Returns:
    float: Acurácia do modelo.
    str: Relatório de classificação.
    """
    y_pred = modelo.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['Não Cancelou', 'Cancelou'])

    print(f"Taxa de acerto no conjunto de teste: {accuracy:.2f}")
    print("Relatório de Classificação:")
    print(report)

    return accuracy, report

def visualizar_arvore(modelo, feature_names, class_names, arquivo_saida="decision_tree"):
    """
    Exporta e visualiza a árvore de decisão utilizando Graphviz.

    Parameters:
    modelo (DecisionTreeClassifier): Modelo treinado.
    feature_names (list): Lista de nomes das características.
    class_names (list): Lista de nomes das classes.
    arquivo_saida (str): Nome do arquivo de saída para a árvore.
    """
    dot_data = export_graphviz(
        modelo,
        out_file=None,
        feature_names=feature_names,
        class_names=class_names,
        filled=True,
        rounded=True,
        special_characters=True
    )

    graph = graphviz.Source(dot_data)
    graph.render(arquivo_saida)
    print(f"Árvore de decisão salva como {arquivo_saida}.pdf")
    graph.view()

def main():
    """
    Função principal que executa todas as etapas do projeto.
    """
    # URL do dataset
    url = 'https://github.com/daniel-usp/MachineLearning/raw/main/04%20-%20DecisionTrees/assinatura.xlsx'

    # Passo 1: Carregar os dados
    dados = carregar_dados(url)

    # Passo 2: Pré-processar os dados
    X, y = preprocessar_dados(dados, 'cancel')

    # Passo 3: Dividir os dados
    X_train, X_test, y_train, y_test = dividir_dados(X, y)

    # Passo 4: Ajustar hiperparâmetros
    melhores_params, melhor_acuracia = ajustar_hiperparametros(X_train, y_train)

    # Passo 5: Treinar o modelo
    modelo = treinar_modelo(X_train, y_train, melhores_params)

    # Passo 6: Avaliar o modelo
    acuracia, relatorio = avaliar_modelo(modelo, X_test, y_test)

    # Passo 7: Visualizar a árvore de decisão
    visualizar_arvore(modelo, X.columns, ['Não Cancelou', 'Cancelou'])

if __name__ == "__main__":
    main()


Dados carregados com sucesso.
Dados pré-processados.
Dados divididos: 1399 para treino e 600 para teste.
Melhores parâmetros encontrados: {'max_depth': 3, 'splitter': 'random'}
Melhor acurácia na validação cruzada: 0.78
Modelo treinado com sucesso.
Taxa de acerto no conjunto de teste: 0.80
Relatório de Classificação:
              precision    recall  f1-score   support

Não Cancelou       0.84      0.91      0.87       457
    Cancelou       0.61      0.44      0.51       143

    accuracy                           0.80       600
   macro avg       0.72      0.68      0.69       600
weighted avg       0.78      0.80      0.79       600

Árvore de decisão salva como decision_tree.pdf


In [1]:
# Importar bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import graphviz

# Carregar o banco de dados
dados = pd.read_excel('https://github.com/daniel-usp/MachineLearning/raw/main/04%20-%20DecisionTrees/assinatura.xlsx')

# Definir X e y
y = dados['cancel']
X = dados.drop('cancel', axis=1)

# Separar amostra treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir a grade de parâmetros
param_grid = {
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15],
    'splitter': ['best', 'random']
}

# Realizar a busca em grade (grid search) para encontrar os melhores parâmetros
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Obter os melhores parâmetros
best_params = grid_search.best_params_
best_max_depth = best_params['max_depth']
best_splitter = best_params['splitter']
best_accuracy = grid_search.best_score_

print(f"O melhor valor de max_depth é: {best_max_depth}")
print(f"O melhor valor de splitter é: {best_splitter}")
print(f"A melhor acurácia durante a validação cruzada é: {best_accuracy:.2f}")

# Treinar o modelo de Árvore de Decisão com os melhores parâmetros
tree = DecisionTreeClassifier(max_depth=best_max_depth, splitter=best_splitter, random_state=42)
tree.fit(X_train, y_train)

# Prever y no conjunto de teste
y_pred = tree.predict(X_test)

# Avaliar a taxa de acerto fora da amostra
accuracy = accuracy_score(y_test, y_pred)
print(f"Taxa de acerto fora da amostra: {accuracy:.2f}")

# Gerar o relatório de classificação
report = classification_report(y_test, y_pred, target_names=['0', '1'])
print(report)


# Exportar a árvore de decisão para o formato Graphviz
dot_data = export_graphviz(
    tree,
    out_file=None,
    feature_names=X.columns,
    class_names=['0', '1'],
    filled=True,
    rounded=True,
    special_characters=True
)

# Visualizar a árvore de decisão com Graphviz
graph = graphviz.Source(dot_data)
graph.render("decision_tree")

# Exibir a árvore renderizada
graph.view()


O melhor valor de max_depth é: 3
O melhor valor de splitter é: random
A melhor acurácia durante a validação cruzada é: 0.78
Taxa de acerto fora da amostra: 0.80
              precision    recall  f1-score   support

           0       0.84      0.91      0.87       457
           1       0.61      0.44      0.51       143

    accuracy                           0.80       600
   macro avg       0.72      0.68      0.69       600
weighted avg       0.78      0.80      0.79       600



'decision_tree.pdf'